# Strategy/Schemes

### 1. 消息面-AI文本分析

Process: 数据获取，数据加工，信息提取，AI提示词工程，DeepSeek-AI量化Agent，AI量化信息提取 [(Source)](https://blog.csdn.net/weixin_39521144/article/details/145424996)

![image.png](attachment:image.png)

In [ ]:
# 导入库
import tushare as ts
from datetime import datetime, timedelta
import os
import json
from openai import OpenAI

# 参数设置
date = '20240101'

# AI人设提示词工程
system_prompt = {
    "角色设定": "DeepSeek金融文本分析与市场预测引擎（沪深300专用）",
    "核心能力": [
        "金融文本深度语义理解",
        "多维度市场环境评估",
        "金融工程量化建模",
        "风险收益动态预测"
    ],
    "输入规范": {
        "数据类型": [
            "央行货币政策文本",
            "监管政策文本",
            "宏观经济新闻",
            "市场分析报告",
            "A股相关新闻",
            "全球市场资讯"
        ],
        "时间控制": {
            "T日锁定": "自动提取输入文本最大日期作为基准日",
            "有效期判定": "文本信息时效性评估",
            "预测周期": "向前12个月滚动预测",
            "数据屏障": "严格隔离T+1日及之后的信息影响"
        }
    },
    "分析框架": {
        "文本特征提取": {
            "经济环境评估": [
                "货币政策环境（宽松/中性/紧缩）",
                "财政政策取向（积极/稳健/保守）",
                "监管政策导向（放松/维持/收紧）",
                "产业政策重点（支持/限制行业）"
            ],
            "市场环境分析": [
                "流动性状况（充裕/适度/紧张）",
                "风险偏好（追逐/平衡/规避）",
                "市场情绪（乐观/中性/悲观）",
                "估值水平（高估/合理/低估）"
            ],
            "风险因素识别": [
                "政策风险（政策转向/监管加强）",
                "经济风险（增长放缓/通胀压力）",
                "市场风险（估值/杠杆/流动性）",
                "外部风险（地缘政治/全球经济）"
            ]
        },
        "量化建模层": {
            "收益率预测模型": {
                "基础模型": [
                    "政策敏感度模型（PSM）",
                    "市场情绪指数（MSI）",
                    "宏观环境评分（MES）"
                ],
                "预期收益计算": {
                    "基准收益": "无风险利率 + 市场风险溢价",
                    "政策调整": "PSM * 政策影响系数",
                    "情绪调整": "MSI * 情绪弹性系数",
                    "环境调整": "MES * 环境敏感度"
                },
                "收益率区间": {
                    "乐观情景": "基准 + 上行调整",
                    "基准情景": "基准 + 中性调整",
                    "悲观情景": "基准 + 下行调整"
                }
            },
            "波动率建模": {
                "基础指标": [
                    "历史波动率基准",
                    "隐含波动率参考",
                    "政策不确定性指数"
                ],
                "调整因子": {
                    "政策风险": "政策不确定性指数变化",
                    "市场情绪": "情绪波动指数",
                    "流动性": "流动性压力指数",
                    "外部冲击": "外部风险溢出系数"
                },
                "波动率计算": {
                    "基础波动": "历史波动率 * (1 + 市场环境调整)",
                    "政策调整": "基础波动 * (1 + 政策风险系数)",
                    "情绪调整": "政策调整后 * (1 + 情绪波动系数)",
                    "最终波动": "情绪调整后 * (1 + 外部风险系数)"
                }
            }
        },
        "预测集成系统": {
            "收益率预测": {
                "模型权重": {
                    "政策导向": "0.4",
                    "市场环境": "0.3",
                    "情绪因子": "0.3"
                },
                "情景权重": {
                    "乐观情景": "0.2",
                    "基准情景": "0.6",
                    "悲观情景": "0.2"
                }
            },
            "波动率预测": {
                "因子权重": {
                    "基础波动": "0.4",
                    "政策风险": "0.3",
                    "市场情绪": "0.2",
                    "外部冲击": "0.1"
                }
            }
        }
    },
    "输出规范": {
        "强制格式": {
            "结构定义": {
                "数据日期": "string（YYYY-MM-DD）",
                "预期收益率": "float（年化，[-30.0,50.0]）",
                "预期波动率": "float（年化，[15.0,80.0]）",
                "环境评分": {
                    "政策环境": "float（[-1.0,1.0]）",
                    "市场环境": "float（[-1.0,1.0]）",
                    "风险环境": "float（[-1.0,1.0]）"
                },
                "预测置信度": "float（[0.0,1.0]）",
                "AI分析报告": "string（Markdown格式）"
            }
        },
        "报告结构": {
            "必需章节": [
                "宏观环境综述",
                "政策导向分析",
                "市场环境评估",
                "风险因素分析",
                "收益率预测逻辑",
                "波动率预测依据",
                "重要风险提示"
            ]
        }
    },
    "风控体系": {
        "文本质量控制": [
            "信息可靠性验证",
            "来源可信度评估",
            "信息时效性检查"
        ],
        "预测可靠性": [
            "模型置信度评估",
            "预测偏差监控",
            "异常预警机制"
        ],
        "风险提示": [
            "预测不确定性说明",
            "潜在风险因素",
            "重要免责声明"
        ]
    }
}

def AI_fin_quanter_text(date, system_prompt):
    # 初始话参数
    pro = ts.pro_api('ts_api')
    api_key = "sk-api"

    # 计算时间区间
    date_obj = datetime.strptime(date, '%Y%m%d')
    end_date = date_obj.strftime('%Y-%m-%d %H:%M:%S')
    start_date = (date_obj - timedelta(days=3)).strftime('%Y-%m-%d %H:%M:%S')

    # 获取raw数据
    df = pro.news(start_date=start_date, end_date=end_date, fields=["datetime","content","channels"])
    df = df.sort_values('datetime', ascending=False)

    # 数据处理
    def json_format(df):
        """
        将DataFrame转换为JSON字符串，处理空数据情况
        
        Args:
            df: pandas DataFrame对象
            
        Returns:
            str: JSON字符串或"无"（当数据为空时）
        """
        if df.empty:
            return "无"
            
        try:
            json_str = df.to_json(
                orient='records',          # 使用records格式，每行数据作为一个对象
                force_ascii=False,         # 支持非ASCII字符（如中文）
                date_format='iso',         # 标准化日期格式
                double_precision=4         # 控制数字精度，避免过长
            )
            return json_str
        except Exception as e:
            print(f"JSON转换错误: {str(e)}")
            return "无"
    # 宏观信息
    df_macro = df[['datetime', 'content']].loc[df['channels'] == '宏观'].head(5)
    # 央行信息
    df_CB = df[['datetime', 'content']].loc[df['channels'] == '央行'].head(5)
    # 美国焦点信息
    df_America = df[['datetime', 'content']].loc[(df['content'].str.contains('美国'))&(df['channels'] == '焦点')].head(5)
    # 市场信息
    df_markets = df[['datetime', 'content']].loc[df['channels'] == '市场'].head(5)
    # A股信息
    df_Astocks = df[['datetime', 'content']].loc[df['channels'] == 'A股'].head(5)

    # json化处理
    df_macro = json_format(df_macro)
    df_CB = json_format(df_CB)
    df_America = json_format(df_America)
    df_markets = json_format(df_markets)
    df_Astocks = json_format(df_Astocks)

    # Messages构建
    messages = [
        {'role': 'system', 'content': json.dumps(system_prompt, ensure_ascii=False)},
        {'role': 'user', 'content': "宏观信息:"+df_macro},
        {'role': 'user', 'content': "央行信息:"+df_CB},
        {'role': 'user', 'content': "美国焦点信息:"+df_America},
        {'role': 'user', 'content': "市场信息:"+df_markets},
        {'role': 'user', 'content': "A股信息:"+df_Astocks},
    ]

    # 构建AI Agent
    client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")

    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
        stream=False
    )

    # response格式处理
    # 提取content内容
    content = response.choices[0].message.content

    # 格式处理
    content = content[content.index('{'):]
    content = content[:content.rindex('}')+1]

    content_dict = json.loads(content)

    return content_dict


In [ ]:
# run 
if __name__ == '__main__':
    content_dict = AI_fin_quanter_text('20230101', system_prompt)
    print(content_dict)

输出的content_dict主要包括要素：

1. 数据日期。是输入数据中的最大日期，也是模型预测认知的约束日期；
1. 预期收益率；
1. 预期波动率；
1. 环境评分；
1. 预测置信度；
1. AI量化投研报告。值的格式即为MD格式的投研报告。

"20230101"的运行结果展示：

In [ ]:
{
    "数据日期": "2022-12-31",
    "预期收益率": 12.5,
    "预期波动率": 22.8,
    "环境评分": {
        "政策环境": 0.3,
        "市场环境": -0.2,
        "风险环境": -0.4
    },
    "预测置信度": 0.75,
    "AI量化投研报告": "## 1. 宏观环境综述

### 1.1 经济基本面
国内经济呈现稳步复苏态势，基建投资持续发力，长庆油田年产油气突破6500万吨创历史新高，社会汽车保有量超3亿辆显示消费潜力，广东高速公路建设等基建投资持续推进，显示经济韧性。

### 1.2 外部环境
美股2022年表现创14年来最差，但高盛预测美国或将实现软着陆。中美关系方面存在军事摩擦，需持续关注地缘政治风险。

## 2. 政策导向分析

### 2.1 货币政策
全球货币政策仍处于收紧周期，欧央行明确表态将继续加息以抑制通胀，但日本央行考虑调整通胀预期，可能暗示政策转向。国内政策预计将保持稳健。

### 2.2 产业政策
新能源、科技创新等领域政策支持力度不减，跨境金融领域监管趋严，显示政策既重视发展又注重防范风险。

## 3. 市场环境评估

### 3.1 流动性状况
年末市场流动性整体平稳，但需关注美联储持续加息对全球流动性的影响。

### 3.2 市场情绪
机构投资者普遍对2023年持谨慎乐观态度，私募基金年末出现反思潮，市场情绪趋于理性。

## 4. 风险因素分析

### 4.1 短期风险
- 全球货币政策收紧延续
- 地缘政治冲突不确定性
- 年初解禁压力（如宁德时代431.8亿解禁）

### 4.2 中期风险
- 经济复苏进程的不确定性
- 外部需求走弱影响
- 产业政策调整风险

## 5. 收益率预测逻辑
基于当前市场环境，预计沪深300指数未来12个月收益率为12.5%，主要考虑：
1. 经济基本面企稳回升
2. 政策环境总体偏暖
3. 估值处于历史较低水平
4. 外资配置意愿回升

## 6. 波动率预测依据
预期波动率22.8%，高于历史均值，原因包括：
1. 全球货币政策不确定性
2. 地缘政治风险持续
3. 经济复苏过程中的波动
4. 市场结构性机会与风险并存

## 7. 重要风险提示
1. 本预测基于当前可获得信息，未来事件演变可能导致预测调整
2. 货币政策转向风险
3. 地缘政治冲突升级风险
4. 产业政策调整风险
5. 市场流动性风险"
}
